# Uncertain Numbers
### Tristan Larkin

In [3]:
import sympy
import numpy

This class will assume Gaussian error and will use the sympy module for symbolic representation for error propagation. Uncertain numbers have two parts, a value $\mu$ and an uncertainty $\sigma$.

In [103]:
class UncertainNumber:
    def __init__(self, value:float, uncertainty:float, symbol:str="x") -> None:
        self.value = value
        self.uncertainty = uncertainty
        self.sym = sympy.Symbol(symbol)
        self.unc_equation = self.sym
    
    def gen_un(*args) -> tuple['UncertainNumber', sympy.Symbol]:
        un = UncertainNumber(*args)
        return (un, un.sym)

    # def __format__(self, __format_spec: str) -> str:
    #     ...

    def __repr__(self) -> str:
        return f"{self.value} ± {self.uncertainty}"
    
    def parts(self) -> tuple[float,float]:
        return (self.value, self.uncertainty)

    def w_avg(xs:list['UncertainNumber']) -> 'UncertainNumber':
        values = [n.value for n in xs]
        sigmas = [n.uncertainty for n in xs]
        weights = [1/(sigma**2) for sigma in sigmas]
        den = sum(weights)

        new_value = sum([w*x for w,x in zip(weights, values)])/den
        new_uncertainty = numpy.sqrt(1/den)
        return UncertainNumber(new_value, new_uncertainty)

    def propagate(func:sympy.Expr, ns:list['UncertainNumber']):
        errors = [sympy.Derivative(func, n.sym) for n in ns]
        res = 0
        for e,n in zip(errors,ns):
            res = res + (e*n.uncertainty)**2
        return sympy.sqrt(res)


In [97]:
sN = sympy.Symbol("S_N")
sT = sympy.Symbol("S_\\tau")
st = sympy.Symbol("S_t")
n0, t = UncertainNumber.gen_un(1, 0, 't')
n1, N0 = UncertainNumber.gen_un(1, sN, 'N_0')
n2, T = UncertainNumber.gen_un(1, sT, '\\tau')

N = N0 * sympy.exp(-t/T)

z = UncertainNumber.propagate(N, [n0, n1, n2])
z



sqrt(S_N**2*Derivative(N_0*exp(-t/\tau), N_0)**2 + S_\tau**2*Derivative(N_0*exp(-t/\tau), \tau)**2)

In [104]:
values = [0.8971,0.8941,0.8929,0.8920,0.881,0.8924,0.8937,0.8958]
uncertainties = [0.0021,0.0014,0.0016,0.0044,0.009,0.0032,0.0048,0.0045]
uns = [UncertainNumber(v, u) for v,u in zip(values, uncertainties)]
print(uns)

print(UncertainNumber.w_avg(uns))

[0.8971 ± 0.0021, 0.8941 ± 0.0014, 0.8929 ± 0.0016, 0.892 ± 0.0044, 0.881 ± 0.009, 0.8924 ± 0.0032, 0.8937 ± 0.0048, 0.8958 ± 0.0045]
0.8939856002768972 ± 0.000850632110379561
